# Rates

## Onderhoudsfunctie

Lees data in en voeg een groepering toe van rate names naar 'currency group'.

Het inlezen en groeperen van de data moet een aparte routine worden

In [1]:
import pandas as pd
import json

In [2]:
date_cols = ["rate_dt","input_dt","actual_dt",'start_term_dt','maturity_term_dt']
keys = ['curve_type','rate_name','rate_type','ccy','time_band']

df = pd.read_csv(r'data\rates_data.csv', 
                 parse_dates = date_cols, 
                 dtype ={"int_basis":'S10', "exchange_name": 'S10'})

In [3]:
# Selectie van range van de laatste 20 dagen
N = 20

df_nd = df[df.rate_dt >= df['rate_dt'].unique()[-N]].copy()

# Toevoegen van gemiddelde en standaard afwijking - daarna berekenen van z-score

df_gr = df_nd.groupby(keys).agg( {'mid' : ['mean', 'std']}) 
df_gr = df_gr.reset_index()

In [4]:
df_gr.head()

curve_type     rate_name   rate_type  ccy time_band        mid          
                                                             mean       std
0  Capital Pr  AT0000A001X2  End of Day  EUR    ACTUAL  103.09002  0.105877
1  Capital Pr  AT0000A0U3T4  End of Day  EUR    ACTUAL  107.98326  0.114234
2  Capital Pr  AT0000A105W3  End of Day  EUR    ACTUAL  107.06419  0.103418
3  Capital Pr  AT0000A185T1  End of Day  EUR    ACTUAL  109.22595  0.115914
4  Capital Pr  AT0000A1FAP5  End of Day  EUR    ACTUAL  109.32499  0.133820

In [5]:
def ccy_group (currency):
    ''' 
    Map currencies in A- B- and C-currencies 
    
    input:
    currency       - string : 3 digit currency code
    
    Output:
    currency group - string    
    '''
    
    a_ccy = ['EUR','GBP','USD','CHF']
    b_ccy = ['NOK','NZD','SEK','CZK']
    # All others are c-ccy's
    
    if any(currency in x for x in a_ccy):
        return 'A-currency'
    if any(currency in x for x in b_ccy):
        return 'B-currency'
    else:
        return 'C-currency'

# Map currencies in currency groups
df_gr['ccy_group'] = df_gr['ccy'].map(ccy_group)

In [6]:
# Select unique list of ccy's and ccy groups
df_ccylist = df_gr[['ccy_group','ccy']].drop_duplicates().copy()
df_ccylist.columns = ['ccy_group','ccy']

In [7]:
# Write to data
file = r'data\ccy_group.json'
r = df_ccylist.to_json(file)